<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain langchain-core langchain-community --quiet
!pip install sentence_transformers FlagEmbedding chromadb --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
import chromadb
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}, #gpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions/questions_ver2.txt", names=['question'])
company = "COCA COLA CO"
questions['question_name'] = questions['question'].str.replace('company', company)

# Approach #1

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as NOT part of the question
* Number of retrieved parent chunks = 2
* Distance function - Cosine similarity

In [8]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": "cosine"})
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb'
    )
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [9]:
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS})

In [10]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        'context_cosine': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [03:35<00:00,  6.16s/it]


In [11]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach1.json')

# Approach #2

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as part of the question
* Number of retrieved parent chunks = 2
* Distance function - Cosine similarity

In [12]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        'context_cosine_name': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:44<00:00,  3.00s/it]


In [13]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach2.json')

# Approach #3

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as NOT part of the question
* Number of retrieved parent chunks = 2
* Distance function - Inner product

In [14]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": "ip"})
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb'
    )
vectorstore.persist()

In [15]:
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS})

In [16]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        'context_ip': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:47<00:00,  3.07s/it]


In [17]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach3.json')

# Approach #4

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as part of the question
* Number of retrieved parent chunks = 2
* Distance function - Inner product

In [18]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        'context_ip_name': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:31<00:00,  2.61s/it]


In [19]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach4.json')

# Approach #5

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as NOT part of the question
* Number of retrieved parent chunks = 2
* Distance function - Squared L2

In [20]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": "l2"})
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb'
    )
vectorstore.persist()

In [21]:
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS})

In [22]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        'context_l2': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:48<00:00,  3.09s/it]


In [23]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach5.json')

# Approach #6

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as part of the question
* Number of retrieved parent chunks = 2
* Distance function - Squared L2

In [24]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        'context_l2_name': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:29<00:00,  2.55s/it]


In [25]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach6.json')